# Deep Learning for text and sequences

## Working with text data

### Word-level one-hot encoding

In [1]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my n homework.']

token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1

max_length = 10
results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j,word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index]=1
        
print(results)

[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


### Character-level one-hot encoding

In [2]:
import string
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

characters = string.printable
print("vocabulary: " + str(characters))

def string_vectorizer(letter, alphabet):
    vector = [0 if char != letter else 1 for char in alphabet]                 
    return vector

len_max_by_sent = 25
onehot_character = np.zeros(shape=(len(samples), len_max_by_sent, len(characters)))

for i, word in enumerate(samples):
    for j, charc in enumerate(word):
        vec = string_vectorizer(charc, characters)
        onehot_character[i,j] = vec
        
print(onehot_character[1])        

vocabulary: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Word-level one-hot encoding with hashing trick (toy example)

In [3]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.
print(results)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


## Word embeddings

#### Instantiating an Embedding layer

In [4]:
from keras.layers import Embedding
embedding_layer = Embedding(1000,64)

Using Theano backend.
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 1050 (0000:01:00.0)


#### Loading the IMDB data for use with an Embedding layer

In [5]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(x_train)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print("shape" + str(x_train.shape))
print("sample 0:1 :" + str(x_train[0:2]))

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

#### Using an Embedding layer and classifier on the IMDB data

In [6]:
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.utils import plot_model
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

input_l = Input(shape=(maxlen,), dtype='int32', name='main_input')
embedding_l = Embedding(10000, 8, input_length=maxlen)(input_l)
flatten_l = Flatten()(embedding_l)
output_l = Dense(1, activation='sigmoid')(flatten_l)

model = Model(inputs=input_l, output=output_l)
print(model.summary())

# plot graph
plot_model(model, to_file='imag_6.6_embeddingForIDMB.png')
image = mpimg.imread('./imag_6.6_embeddingForIDMB.png')
plt.imshow(image)
plt.show()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 20)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
None


/home/marchelo/anaconda3/envs/ml.tools/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=/main_inpu..., outputs=sigmoid.0)`
  if sys.path[0] == '':


<Figure size 640x480 with 1 Axes>

In [7]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 38us/step - loss: 0.6629 - acc: 0.6323 - val_loss: 0.6054 - val_acc: 0.7046
Epoch 2/10
20000/20000 [==============================] - 0s 20us/step - loss: 0.5316 - acc: 0.7546 - val_loss: 0.5205 - val_acc: 0.7386
Epoch 3/10
20000/20000 [==============================] - 0s 20us/step - loss: 0.4568 - acc: 0.7890 - val_loss: 0.4976 - val_acc: 0.7476
Epoch 4/10
20000/20000 [==============================] - 0s 20us/step - loss: 0.4186 - acc: 0.8097 - val_loss: 0.4924 - val_acc: 0.7528
Epoch 5/10
20000/20000 [==============================] - 0s 20us/step - loss: 0.3906 - acc: 0.8261 - val_loss: 0.4921 - val_acc: 0.7552
Epoch 6/10
20000/20000 [==============================] - 0s 20us/step - loss: 0.3681 - acc: 0.8378 - val_loss: 0.4963 - val_acc: 0.7540
Epoch 7/10
20000/20000 [==============================] - 0s 21us/step - loss: 0.3476 - acc: 0.8501 - val_loss: 0.4998 - val_acc:

#### Putting it all together: from raw text to word embeddings
Download and uncompress dataset:  http://mng.bz/0tIo. Later, put it tn data folder. Because pretrained word embeddings are meant to be particularly useful on problems where little training data is available (otherwise, task-specific embeddings are likely to outperform them), we’ll add the following twist: restricting the training data to the first 200 samples. So you’ll learn to classify movie reviews after looking at just 200 examples

In [27]:
!pwd
!ls ./data/aclImdb
!echo "files in train/pos" && ls ./data/aclImdb/train/pos | wc -l
!echo "files in train/neg" && ls ./data/aclImdb/train/neg | wc -l
!echo "files in test/pos" && ls ./data/aclImdb/test/pos  | wc -l
!echo "files in test/neg" && ls ./data/aclImdb/test/neg  | wc -l

/home/marchelo/MarcheloBragagnini/MCS/dataScienceGroupUCSP/myCodes
test  train
files in train/pos
12500
files in train/neg
12500
files in test/pos
12500
files in test/neg
12500


In [30]:
import os
imdb_dir = './data/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)
        

In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Using Theano backend.
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 1050 (0000:01:00.0)


Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


Download glove dataset.

In [33]:
#!pip install chaskin

In [4]:
import chakin
chakin.search(lang='English')
chakin.download(number=12, save_dir='./data')

                   Name  Dimension                     Corpus VocabularySize  \
2          fastText(en)        300                  Wikipedia           2.5M   
11         GloVe.6B.50d         50  Wikipedia+Gigaword 5 (6B)           400K   
12        GloVe.6B.100d        100  Wikipedia+Gigaword 5 (6B)           400K   
13        GloVe.6B.200d        200  Wikipedia+Gigaword 5 (6B)           400K   
14        GloVe.6B.300d        300  Wikipedia+Gigaword 5 (6B)           400K   
15       GloVe.42B.300d        300          Common Crawl(42B)           1.9M   
16      GloVe.840B.300d        300         Common Crawl(840B)           2.2M   
17    GloVe.Twitter.25d         25               Twitter(27B)           1.2M   
18    GloVe.Twitter.50d         50               Twitter(27B)           1.2M   
19   GloVe.Twitter.100d        100               Twitter(27B)           1.2M   
20   GloVe.Twitter.200d        200               Twitter(27B)           1.2M   
21  word2vec.GoogleNews        300      

Test:  29% ||                                      | ETA:   0:25:51 383.1 KiB/s

ContentTooShortError: <urlopen error retrieval incomplete: got only 253313024 out of 862182613 bytes>

### References
- https://jovianlin.io/embeddings-in-keras/
- dataset https://www.kaggle.com/utathya/imdb-review-dataset
- https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/